# Constant False Alarm Rate Testing

In [ ]:
# Importing all needed libraries and functions
import numpy as np
import time
from tqdm import tqdm

# Plotting
import matplotlib.pyplot as plt

# Custom functions
from generic_functions import *
from multivariate_images_tools import *
from change_detection_functions import *
from monte_carlo_tools import *

## Simulation parameters

In [ ]:
# General parameters
p = 10    # Dimension of vectors
N = 25    # Number of samples
T = 2     # Number of dates

# Enable parallel processing (or not)
enable_multi = False
# These two variables serves to split the original image into sub-images to be treated in parallel
# In general the optimal parameters are obtained for 
# number_of_threads = number of cores on the machine
number_of_threads = 4


# Number of Monte Carlo trials to compute histograms
number_of_trials = 4000  
number_of_bins_histogram = 30

# List of statistics to compute
statistic_list = [covariance_equality_glrt_gaussian_statistic, 
                 covariance_equality_t1_gaussian_statistic,
                  covariance_equality_Wald_gaussian_statistic,
                 complex_Hotelling_Lawley_trace_statistic,
                  scale_and_shape_equality_robust_statistic,
                  Kullback_Leibler_statistic,
                  Similarity_measure_Meng_Liu,
                  Natural_Geodesic_statistic,
                Natural_student_t_distance,
                Wasserstein_Gaussian_statistic,
                 Wasserstein_robust_statistic]
statistic_names = [r'Gaussian GLRT', 
                   r't1 statistic',
                   r'Wald statistic',
                   r'HTL statistic',
                   r'MT statistic',
                  r'Kullback Leibler statistic',
                   r'MLL statistic',
                   r'Natural Gaussian Geodesic statistic',
                    r'Natural_student_t_distance',
                   r'Wasserstein Gaussian statistic',
                  r'Wasserstein robust statistic']
args_list = ['log', 'linear', 'log', None, (0.01, 15, 'log'), None, (0.01,15), None, (3,0.01,15),  None, (0.01,15)]
number_of_statistics = len(statistic_list)

## Testing Contant False Alarm property by comparing histograms of statistics with different covariance matrices

In [ ]:
# Parameters for Multivariate Complex Gaussian distribution
𝛍 = np.zeros(p)
pseudo_𝚺 = 0
ρ_vec = [0.1, 0.5, 0.9]  # List of Toeplitz matrices
τ_vec = [10,20,30]

# Setting useful variables for the Monte-Carlo
data_generation_function = generate_time_series_multivariate_vector
generation_function = wrapper_multivariate_complex_normal_samples
function_to_compute = compute_several_statistics
function_args = [statistic_list, args_list]

In [ ]:
# Doing the Monte Carlo simulation
λ = np.zeros((number_of_trials, len(statistic_list), len(ρ_vec)))
for i_ρ, ρ in enumerate(ρ_vec): # Iterate for each covariance matrix
        print("Doing Monte Carlo %d of %d" % (i_ρ+1, len(ρ_vec)))
        
        # Generating Covariance matrix with given ρ
        𝚺 = τ_vec[i_ρ]*ToeplitzMatrix(ρ, p)
        
        # Generating parameters to pass to the Monte-Carlo function: since
        # the series is homogeneous, we repeat T times
        data_args_list = [[𝛍, 𝚺, N, pseudo_𝚺]]*T 
        data_generation_args = [p, N, T, generation_function, data_args_list]

        # Computing the Monte-Carlo simulation for this covariance matrix
        λ[:,:,i_ρ] = np.array(compute_monte_carlo_parallel(data_generation_function, data_generation_args, 
                                        function_to_compute, function_args, 
                                        number_of_trials, multi=enable_multi, number_of_threads=number_of_threads,
                                        progressbar=not enable_multi))

In [ ]:
# Plotting
for i_Λ, Λ in enumerate(statistic_names): # A figure by statistic
    plt.figure(figsize=(12, 7), dpi=80, facecolor='w')
    ax = plt.gca()
    for i_ρ, ρ in enumerate(ρ_vec): # Plotting the different histograms for each value of ρ
        # Sometimes infinite value can appear due to overflow, we skip them
        λ_to_plot = λ[:,i_Λ, i_ρ]
        λ_to_plot = λ_to_plot[λ_to_plot!=np.inf]
        plt.hist(λ_to_plot, number_of_bins_histogram, 
                     label=r'$\rho=%.2f$' % ρ, alpha=0.5, density=True,
                     edgecolor='black', linewidth=0.5)
    plt.legend()
    plt.xlabel(r'$\lambda$')
    plt.ylabel(r'PDF')
    plt.title(r'Martix CFAR property for %s' % Λ)
plt.show()